In [1]:
import os
import requests
import numpy as np
import pandas as pd

/Users/jaeminiman/anaconda3/envs/pytorch/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 한국은행 Open API 서비스

#### 100대 통계지표

In [25]:
url = 'https://ecos.bok.or.kr/api/KeyStatisticList/HLOXWJTZRBR3KCAG5JU3/json/kr/1/101'


In [26]:
# API 호출
response = requests.get(url)

# 응답 데이터 확인
if response.status_code == 200:
    data = response.json()  # JSON 데이터를 Python 딕셔너리로 변환
    print(data)
else:
    print(f"Error: {response.status_code}, {response.text}")


{'KeyStatisticList': {'list_total_count': 101, 'row_count': 101, 'row': [{'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '한국은행 기준금리', 'DATA_VALUE': '3', 'CYCLE': '20250102', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '콜금리(익일물)', 'DATA_VALUE': '3.029', 'CYCLE': '20250102', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': 'KORIBOR(3개월)', 'DATA_VALUE': '3.27', 'CYCLE': '20250103', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': 'CD수익률(91일)', 'DATA_VALUE': '3.15', 'CYCLE': '20250103', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '통안증권수익률(364일)', 'DATA_VALUE': '2.646', 'CYCLE': '20250103', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '국고채수익률(3년)', 'DATA_VALUE': '2.482', 'CYCLE': '20250103', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '국고채수익률(5년)', 'DATA_VALUE': '2.643', 'CYCLE': '20250103', 'UNIT_NAME': '% '}, {'CLASS_NAME': '시장금리', 'KEYSTAT_NAME': '회사채수익률(3년,AA-)', 'DATA_VALUE': '3.17', 'CYCLE': '20250103', 'UNIT_NAME':

In [28]:
keystat_df = pd.DataFrame(data['KeyStatisticList']['row'])

In [29]:
keystat_df.head(3)

,CLASS_NAME,KEYSTAT_NAME,DATA_VALUE,CYCLE,UNIT_NAME
0,시장금리,한국은행 기준금리,3,20250102,%
1,시장금리,콜금리(익일물),3.029,20250102,%
2,시장금리,KORIBOR(3개월),3.27,20250103,%


In [32]:
keystat_df.describe(include='object')

,CLASS_NAME,KEYSTAT_NAME,DATA_VALUE,CYCLE,UNIT_NAME
count,101,101,100,100,95
unique,28,101,100,9,18
top,시장금리,한국은행 기준금리,3,202411,%
freq,8,1,1,39,27


In [33]:
keystat_df['CLASS_NAME'].value_counts()

CLASS_NAME
시장금리          8
생산            7
투자            7
국제수지          5
성장률           5
소비자/생산자 물가    4
통관수출입         4
고용            4
심리지표          4
주식            4
예금/대출금        4
통화량           4
환율            4
노동            3
부동산 가격        3
대외채권/채무       3
인구            3
GDP대비 비율      3
기업경영지표        3
소비            3
소득            2
소득분배지표        2
가계            2
여수신금리         2
경기순환지표        2
채권            2
수출입 물가        2
국제원자재가격       2
Name: count, dtype: int64

In [34]:
keystat_df[keystat_df['CLASS_NAME']=='시장금리']


,CLASS_NAME,KEYSTAT_NAME,DATA_VALUE,CYCLE,UNIT_NAME
0,시장금리,한국은행 기준금리,3,20250102,%
1,시장금리,콜금리(익일물),3.029,20250102,%
2,시장금리,KORIBOR(3개월),3.27,20250103,%
3,시장금리,CD수익률(91일),3.15,20250103,%
4,시장금리,통안증권수익률(364일),2.646,20250103,%
5,시장금리,국고채수익률(3년),2.482,20250103,%
6,시장금리,국고채수익률(5년),2.643,20250103,%
7,시장금리,"회사채수익률(3년,AA-)",3.17,20250103,%


In [35]:
keystat_df.to_csv('keystat.csv')

In [37]:
# 경제성장률
# 소비자물가지수
# 실업률
# 환율
# M2 (통화량)
# 국고채 수익률 (5년)
# 경상수지
# 코스피지수
# 수출금액지수
# 주택매매가격지수
# 자동차판매액지수
# 건설수주액
# 소비자심리지수
# 